In [1]:
import pandas as pd
import xgboost as xgb
import sys,random
import pickle
import os
import numpy as np

In [2]:
if not os.path.exists('featurescore'):
    os.mkdir('featurescore')
if not os.path.exists('model'):
    os.mkdir('model')
if not os.path.exists('preds'):
    os.mkdir('preds')

In [3]:
train_x_d_cols = pd.read_csv('./rank_d_feature_score.csv')
train_x_d_cols = list(train_x_d_cols.iloc[10:810].feature)

In [4]:
%%time
#use rank_d  and rank_nd feature
#load data
train_x_date = pd.read_csv('../../preprocess_data/train_x_date.csv').drop(columns=['id'])
train_x_null = pd.read_csv('../../preprocess_data/train_x_null.csv').drop(columns=['id'])
train_x_int = pd.read_csv('../../preprocess_data/train_x_int.csv').drop(columns=['id','tag']) # valid中无‘tag’
train_x_d = pd.read_csv('../../preprocess_data/train_x_float_rank_d.csv',usecols=train_x_d_cols)
train_x_nd = pd.read_csv('../../preprocess_data/train_x_float_rank_nd.csv').drop(columns=['id'])


train_x = pd.concat([train_x_date,train_x_null,train_x_int,train_x_d,train_x_nd],axis=1,ignore_index=True,copy=False)
train_y = pd.read_csv('../../preprocess_data/train_y_33465.csv')
print('正负样本不均衡',train_y.label.value_counts().to_dict())
dtrain = xgb.DMatrix(train_x.values, label=train_y.values)

valid_date = pd.read_csv('../../preprocess_data/valid_date.csv')
valid_null = pd.read_csv('../../preprocess_data/valid_null.csv').drop(columns=['id'])
valid_int = pd.read_csv('../../preprocess_data/valid_int.csv').drop(columns=['id'])
valid_d = pd.read_csv('../../preprocess_data/valid_float_rank_d.csv',usecols=train_x_d_cols)
valid_nd = pd.read_csv('../../preprocess_data/valid_float_rank_nd.csv').drop(columns=['id'])
valid = pd.concat([valid_date,valid_null,valid_int,valid_d,valid_nd],axis=1,ignore_index=True,copy=False)
# pd.concat后特征名没有了，变成数字了
valid_id = valid.iloc[:,0].values
valid = valid.drop(0,axis=1)
dvalid = xgb.DMatrix(valid.values)

正负样本不均衡 {0.0: 31267, 1.0: 2198}
CPU times: user 10.9 s, sys: 952 ms, total: 11.9 s
Wall time: 13.6 s


In [10]:
def pipeline(dtrain,dtest,test_id,iteration,random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight):
    if max_depth==6:
        num_boost_round = 1200
    elif max_depth==7:
        num_boost_round = 1000
    elif max_depth==8:
        num_boost_round = 600
    
    params={
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'early_stopping_rounds':100,
    'scale_pos_weight': float(len(train_y)-np.sum(train_y.values))/float(np.sum(train_y.values)),  # 负例样本除以正例样本
    'eval_metric': 'auc',
    'gamma':gamma,
    'max_depth':max_depth,
    'lambda':lambd,
    'subsample':subsample,
    'colsample_bytree':colsample_bytree,
    'min_child_weight':min_child_weight, 
    'eta': 0.04,
    'seed':random_seed,
    'nthread':16
        }
    watchlist  = [(dtrain,'train')]
    model = xgb.train(params,dtrain,num_boost_round=num_boost_round,evals=watchlist)
    model.save_model('./model/xgb{0}.model'.format(iteration))
    
    #predict test set
    test_y = model.predict(dtest)
    test_result = pd.DataFrame(test_id,columns=["id"])
    test_result['score'] = test_y
    test_result.to_csv("./preds/xgb{0}.csv".format(iteration),index=None,encoding='utf-8')
    
    #get feature score
    feature_score = model.get_fscore()
    feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
    fs = []
    for (key,value) in feature_score:
        fs.append("{0},{1}\n".format(key,value))
    
    with open('./featurescore/feature_score_{0}.csv'.format(iteration),'w') as f:
        f.writelines("feature,score\n")
        f.writelines(fs)
    

In [ ]:
%%time
random_seed = list(range(0,10000,100))
gamma = [i/1000.0 for i in range(100,200,2)]
max_depth = [6,7,8]
lambd = list(range(200,400,2))
subsample = [i/1000.0 for i in range(600,700,2)]
colsample_bytree = [i/1000.0 for i in range(250,350,2)]
min_child_weight = [i/1000.0 for i in range(200,300,2)]
random.shuffle(random_seed)
random.shuffle(gamma)
random.shuffle(max_depth)
random.shuffle(lambd)
random.shuffle(subsample)
random.shuffle(colsample_bytree)
random.shuffle(min_child_weight)

with open('params.pkl','wb') as f:
    pickle.dump((random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight),f)

#to reproduce my result, uncomment following lines
"""
with open('params_for_reproducing.pkl','rb') as f:
    random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight = pickle.load(f)    
"""

for i in range(36):
    pipeline(dtrain,dvalid,valid_id,i,random_seed[i],gamma[i],max_depth[i%3],lambd[i],subsample[i],colsample_bytree[i],min_child_weight[i])

[0]	train-auc:0.753544
[1]	train-auc:0.777016
[2]	train-auc:0.783776
[3]	train-auc:0.792683
[4]	train-auc:0.796864
[5]	train-auc:0.800855
[6]	train-auc:0.802563
[7]	train-auc:0.804778
[8]	train-auc:0.805348
[9]	train-auc:0.806454
[10]	train-auc:0.807606
[11]	train-auc:0.809246
[12]	train-auc:0.810549
[13]	train-auc:0.81137
[14]	train-auc:0.812721
[15]	train-auc:0.813653
[16]	train-auc:0.814721
[17]	train-auc:0.816025
[18]	train-auc:0.816991
[19]	train-auc:0.818006
[20]	train-auc:0.818616
[21]	train-auc:0.819692
[22]	train-auc:0.821257
[23]	train-auc:0.822163
[24]	train-auc:0.823103
[25]	train-auc:0.824236
[26]	train-auc:0.825547
[27]	train-auc:0.826904
[28]	train-auc:0.827741
[29]	train-auc:0.828766
[30]	train-auc:0.829858
[31]	train-auc:0.831034
[32]	train-auc:0.832099
[33]	train-auc:0.833041
[34]	train-auc:0.833777
[35]	train-auc:0.834362
[36]	train-auc:0.835193
[37]	train-auc:0.836215
[38]	train-auc:0.836902
[39]	train-auc:0.837739
[40]	train-auc:0.838238
[41]	train-auc:0.838683
[42

[334]	train-auc:0.972416
[335]	train-auc:0.972596
[336]	train-auc:0.972778
[337]	train-auc:0.97289
[338]	train-auc:0.9731
[339]	train-auc:0.973284
[340]	train-auc:0.973554
[341]	train-auc:0.973772
[342]	train-auc:0.973952
[343]	train-auc:0.974091
[344]	train-auc:0.974246
[345]	train-auc:0.97443
[346]	train-auc:0.974588
[347]	train-auc:0.974748
[348]	train-auc:0.974922
[349]	train-auc:0.975092
[350]	train-auc:0.975226
[351]	train-auc:0.975448
[352]	train-auc:0.97564
[353]	train-auc:0.975843
[354]	train-auc:0.976029
[355]	train-auc:0.976147
[356]	train-auc:0.976252
[357]	train-auc:0.976407
[358]	train-auc:0.976544
[359]	train-auc:0.9767
[360]	train-auc:0.976857
[361]	train-auc:0.976974
[362]	train-auc:0.977159
[363]	train-auc:0.977263
[364]	train-auc:0.977404
[365]	train-auc:0.977541
[366]	train-auc:0.977654
[367]	train-auc:0.977746
[368]	train-auc:0.977824
[369]	train-auc:0.977988
[370]	train-auc:0.978147
[371]	train-auc:0.97825
[372]	train-auc:0.97839
[373]	train-auc:0.978465
[374]	tra

[663]	train-auc:0.997007
[664]	train-auc:0.99703
[665]	train-auc:0.997053
[666]	train-auc:0.997069
[667]	train-auc:0.997077
[668]	train-auc:0.997095
[669]	train-auc:0.997116
[670]	train-auc:0.997131
[671]	train-auc:0.997143
[672]	train-auc:0.997161
[673]	train-auc:0.997177
[674]	train-auc:0.997199
[675]	train-auc:0.997227
[676]	train-auc:0.997241
[677]	train-auc:0.997273
[678]	train-auc:0.997286
[679]	train-auc:0.997301
[680]	train-auc:0.997327
[681]	train-auc:0.997347
[682]	train-auc:0.997374
[683]	train-auc:0.997401
[684]	train-auc:0.997419
[685]	train-auc:0.997446
[686]	train-auc:0.997458
[687]	train-auc:0.997471
[688]	train-auc:0.99749
[689]	train-auc:0.997498
[690]	train-auc:0.997521
[691]	train-auc:0.99753
[692]	train-auc:0.997545
[693]	train-auc:0.997559
[694]	train-auc:0.99758
[695]	train-auc:0.997615
[696]	train-auc:0.997615
[697]	train-auc:0.997643
[698]	train-auc:0.997661
[699]	train-auc:0.997674
[700]	train-auc:0.997696
[701]	train-auc:0.997709
[702]	train-auc:0.997727
[703

[993]	train-auc:0.999698
[994]	train-auc:0.9997
[995]	train-auc:0.999702
[996]	train-auc:0.999702
[997]	train-auc:0.999703
[998]	train-auc:0.999704
[999]	train-auc:0.999705
[1000]	train-auc:0.999708
[1001]	train-auc:0.999708
[1002]	train-auc:0.99971
[1003]	train-auc:0.999712
[1004]	train-auc:0.999715
[1005]	train-auc:0.999715
[1006]	train-auc:0.999714
[1007]	train-auc:0.999717
[1008]	train-auc:0.999719
[1009]	train-auc:0.999722
[1010]	train-auc:0.999725
[1011]	train-auc:0.999726
[1012]	train-auc:0.99973
[1013]	train-auc:0.999731
[1014]	train-auc:0.999732
[1015]	train-auc:0.999734
[1016]	train-auc:0.999738
[1017]	train-auc:0.999738
[1018]	train-auc:0.99974
[1019]	train-auc:0.999741
[1020]	train-auc:0.999743
[1021]	train-auc:0.999745
[1022]	train-auc:0.999746
[1023]	train-auc:0.999745
[1024]	train-auc:0.999748
[1025]	train-auc:0.999751
[1026]	train-auc:0.999753
[1027]	train-auc:0.999755
[1028]	train-auc:0.999757
[1029]	train-auc:0.999759
[1030]	train-auc:0.999762
[1031]	train-auc:0.99976

[119]	train-auc:0.939824
[120]	train-auc:0.940432
[121]	train-auc:0.941052
[122]	train-auc:0.941709
[123]	train-auc:0.942335
[124]	train-auc:0.942794
[125]	train-auc:0.943594
[126]	train-auc:0.94412
[127]	train-auc:0.944734
[128]	train-auc:0.945132
[129]	train-auc:0.945751
[130]	train-auc:0.946202
[131]	train-auc:0.946872
[132]	train-auc:0.947308
[133]	train-auc:0.947956
[134]	train-auc:0.948432
[135]	train-auc:0.948988
[136]	train-auc:0.949637
[137]	train-auc:0.950232
[138]	train-auc:0.950736
[139]	train-auc:0.95133
[140]	train-auc:0.951937
[141]	train-auc:0.952482
[142]	train-auc:0.952985
[143]	train-auc:0.953436
[144]	train-auc:0.953746
[145]	train-auc:0.95427
[146]	train-auc:0.954937
[147]	train-auc:0.955466
[148]	train-auc:0.955933
[149]	train-auc:0.956165
[150]	train-auc:0.956694
[151]	train-auc:0.95736
[152]	train-auc:0.957716
[153]	train-auc:0.958034
[154]	train-auc:0.958399
[155]	train-auc:0.958957
[156]	train-auc:0.959327
[157]	train-auc:0.959779
[158]	train-auc:0.960231
[159

...

[554]	train-auc:0.992744
[555]	train-auc:0.992784
[556]	train-auc:0.992795
[557]	train-auc:0.992874
[558]	train-auc:0.992944
[559]	train-auc:0.993037
[560]	train-auc:0.993094
[561]	train-auc:0.99314
[562]	train-auc:0.993174
[563]	train-auc:0.993255
[564]	train-auc:0.99326
[565]	train-auc:0.993292
[566]	train-auc:0.993324
[567]	train-auc:0.993366
[568]	train-auc:0.99343
[569]	train-auc:0.993472
[570]	train-auc:0.993534
[571]	train-auc:0.993577
[572]	train-auc:0.993627
[573]	train-auc:0.993659
[574]	train-auc:0.993705
[575]	train-auc:0.993754
[576]	train-auc:0.993783
[577]	train-auc:0.993836
[578]	train-auc:0.993881
[579]	train-auc:0.993924
[580]	train-auc:0.993955
[581]	train-auc:0.994004
[582]	train-auc:0.99406
[583]	train-auc:0.994104
[584]	train-auc:0.994146
[585]	train-auc:0.994182
[586]	train-auc:0.994223
[587]	train-auc:0.994269
[588]	train-auc:0.994312
[589]	train-auc:0.994349
[590]	train-auc:0.994407
[591]	train-auc:0.994444
[592]	train-auc:0.994509
[593]	train-auc:0.994558
[594

[884]	train-auc:0.999262
[885]	train-auc:0.999265
[886]	train-auc:0.999266
[887]	train-auc:0.999271
[888]	train-auc:0.999277
[889]	train-auc:0.999286
[890]	train-auc:0.999292
[891]	train-auc:0.999303
[892]	train-auc:0.999302
[893]	train-auc:0.999313
[894]	train-auc:0.999317
[895]	train-auc:0.999323
[896]	train-auc:0.99933
[897]	train-auc:0.999336
[898]	train-auc:0.99934
[899]	train-auc:0.999349
[900]	train-auc:0.999352
[901]	train-auc:0.999355
[902]	train-auc:0.999359
[903]	train-auc:0.999365
[904]	train-auc:0.999366
[905]	train-auc:0.999368
[906]	train-auc:0.999373
[907]	train-auc:0.999375
[908]	train-auc:0.999379
[909]	train-auc:0.999383
[910]	train-auc:0.999387
[911]	train-auc:0.999387
[912]	train-auc:0.999391
[913]	train-auc:0.999392
[914]	train-auc:0.999393
[915]	train-auc:0.999394
[916]	train-auc:0.999397
[917]	train-auc:0.9994
[918]	train-auc:0.999405
[919]	train-auc:0.999413
[920]	train-auc:0.999421
[921]	train-auc:0.999425
[922]	train-auc:0.999433
[923]	train-auc:0.999436
[924

[6]	train-auc:0.809043
[7]	train-auc:0.810624
[8]	train-auc:0.811057
[9]	train-auc:0.813497
[10]	train-auc:0.815809
[11]	train-auc:0.816495
[12]	train-auc:0.816932
[13]	train-auc:0.818407
[14]	train-auc:0.820364
[15]	train-auc:0.822118
[16]	train-auc:0.824305
[17]	train-auc:0.825355
[18]	train-auc:0.827364
[19]	train-auc:0.827756
[20]	train-auc:0.829243
[21]	train-auc:0.830198
[22]	train-auc:0.832314
[23]	train-auc:0.833394
[24]	train-auc:0.834221
[25]	train-auc:0.835694
[26]	train-auc:0.836591
[27]	train-auc:0.838126
[28]	train-auc:0.838782
[29]	train-auc:0.84028
[30]	train-auc:0.841357
[31]	train-auc:0.842227
[32]	train-auc:0.843025
[33]	train-auc:0.843795
[34]	train-auc:0.844983
[35]	train-auc:0.84616
[36]	train-auc:0.84714
[37]	train-auc:0.847951
[38]	train-auc:0.848888
[39]	train-auc:0.84982
[40]	train-auc:0.850939
[41]	train-auc:0.851885
[42]	train-auc:0.853155
[43]	train-auc:0.854034
[44]	train-auc:0.854919
[45]	train-auc:0.855429
[46]	train-auc:0.856426
[47]	train-auc:0.857453


[339]	train-auc:0.983336
[340]	train-auc:0.983451
[341]	train-auc:0.983571
[342]	train-auc:0.983711
[343]	train-auc:0.983813
[344]	train-auc:0.983959
[345]	train-auc:0.984118
[346]	train-auc:0.984254
[347]	train-auc:0.984375
[348]	train-auc:0.984534
[349]	train-auc:0.984699
[350]	train-auc:0.984821
[351]	train-auc:0.984982
[352]	train-auc:0.985168
[353]	train-auc:0.985327
[354]	train-auc:0.985474
[355]	train-auc:0.98561
[356]	train-auc:0.985743
[357]	train-auc:0.985846
[358]	train-auc:0.985985
[359]	train-auc:0.986085
[360]	train-auc:0.98623
[361]	train-auc:0.98633
[362]	train-auc:0.986446
[363]	train-auc:0.986564
[364]	train-auc:0.986719
[365]	train-auc:0.986793
[366]	train-auc:0.986887
[367]	train-auc:0.986978
[368]	train-auc:0.987073
[369]	train-auc:0.987196
[370]	train-auc:0.987323
[371]	train-auc:0.987397
[372]	train-auc:0.987543
[373]	train-auc:0.987641
[374]	train-auc:0.987728
[375]	train-auc:0.987794
[376]	train-auc:0.987909
[377]	train-auc:0.988025
[378]	train-auc:0.988105
[37

[72]	train-auc:0.865392
[73]	train-auc:0.866297
[74]	train-auc:0.867003
[75]	train-auc:0.867705
[76]	train-auc:0.86855
[77]	train-auc:0.86911
[78]	train-auc:0.869855
[79]	train-auc:0.870568
[80]	train-auc:0.871366
[81]	train-auc:0.872224
[82]	train-auc:0.87304
[83]	train-auc:0.873912
[84]	train-auc:0.874478
[85]	train-auc:0.875136
[86]	train-auc:0.875886
[87]	train-auc:0.876781
[88]	train-auc:0.877475
[89]	train-auc:0.878161
[90]	train-auc:0.878749
[91]	train-auc:0.879619
[92]	train-auc:0.880602
[93]	train-auc:0.881617
[94]	train-auc:0.881943
[95]	train-auc:0.882452
[96]	train-auc:0.882988
[97]	train-auc:0.883648
[98]	train-auc:0.884191
[99]	train-auc:0.885054
[100]	train-auc:0.885434
[101]	train-auc:0.885767
[102]	train-auc:0.88637
[103]	train-auc:0.887275
[104]	train-auc:0.887857
[105]	train-auc:0.888637
[106]	train-auc:0.889465
[107]	train-auc:0.890032
[108]	train-auc:0.890452
[109]	train-auc:0.891052
[110]	train-auc:0.891744
[111]	train-auc:0.892096
[112]	train-auc:0.892645
[113]	t